# Conhecendo  o dataset

In [ ]:
import pandas as pd
artigo_treino = pd.read_csv('treino.csv')
artigo_teste = pd.read_csv('teste.csv')
artigo_treino.sample(5)

In [ ]:
artigo_teste.iloc[643].title

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

texto = [
        "tenha um bom dia",
        "tenha um péssimo dia",
        "tenha um ótimo dia",
        "tenha um dia ruim"
]

vetorizador = CountVectorizer()
vetorizador.fit(texto)

In [ ]:
print(vetorizador.vocabulary_)

In [ ]:
vetor_bom = vetorizador.transform(["bom"])
print(vetor_bom.toarray())

In [ ]:
vetor_otimo = vetorizador.transform(["ótimo"])
print(vetor_otimo.toarray())

# Link para o arquivo 

http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

In [ ]:
with open("cbow_s300.txt") as f:
    for linha in range(10):
        print(next(f))

In [ ]:
# Instalação da biblioteca gensim
!pip install gensim

In [ ]:
from gensim.models import KeyedVectors

modelo = KeyedVectors.load_word2vec_format("cbow_s300.txt")

In [ ]:
modelo.most_similar("china")

In [ ]:
modelo.most_similar("itália")

In [ ]:
modelo.most_similar(positive=["brasil", "argentina"])

In [ ]:
# nuvens -> nuvem : estrelas -> estrela
# nuvens - nuvem + estrela = estrelas

modelo.most_similar(positive=["nuvens", "estrela"], negative=["nuvem"])

In [ ]:
# vieses de gênero -> https://www.weforum.org/stories/2019/01/ai-isn-t-dangerous-but-human-bias-is/
modelo.most_similar(positive=["médico", "mulher"], negative=["homem"])

## Vetorização de texto


In [ ]:
artigo_treino.title.loc[12]

In [ ]:
# instalação da biblioteca nltk
!pip install nltk

In [ ]:
# Construindo o tokenizador
import nltk
import string
# baixar o modelo punkt (tokenizer) se necessário
nltk.download('punkt')
nltk.download('punkt_tab')


def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerico = []
    
    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.punctuation:
            continue
        # mantém tokens que não são apenas pontuação (inclui números e palavras)
        lista_alfanumerico.append(token_valido)

    return lista_alfanumerico

tokenizador("TExto Exemplo, 1234.")

In [ ]:
# Combinação de vetores por soma
import numpy as np
def combinacao_vetores_por_soma(palavras_numeros):
    vetor_resultante = np.zeros(300,)  # vetor de zeros com dimensão 300
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0"*len(pn)  # substitui números por "0", "00", "000", etc.
                vetor_resultante += modelo.get_vector(pn)
            else:
                vetor_resultante += modelo.get_vector("unknown")     
    
    return vetor_resultante

In [ ]:
palavras_numeros = tokenizador("texto exemplo 123 callll")
vetor_texto = combinacao_vetores_por_soma(palavras_numeros)
print(len(vetor_texto))
print(vetor_texto)

In [ ]:
# Criar função para matrix de vetores
def matriz_vetores(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores_por_soma(palavras_numeros)

    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter=200)
LR.fit(matriz_vetores_treino, artigo_treino.category)

In [ ]:
LR.score(matriz_vetores_teste, artigo_teste.category)

In [ ]:
artigo_teste.category.unique()

In [ ]:
from sklearn.metrics import classification_report

label_prevista = LR.predict(matriz_vetores_teste)

CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

In [ ]:
from sklearn.dummy import DummyClassifier

DC = DummyClassifier()
DC.fit(matriz_vetores_treino, artigo_treino.category)
label_prevista_dc = DC.predict(matriz_vetores_teste)

CR_dummy = classification_report(artigo_teste.category, label_prevista_dc)
print(CR_dummy)

In [ ]:
# Testando o modelo com skip-gram
# http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc#:~:text=stemmiza%C3%A7%C3%A3o%20e%20outras.-,Word2Vec,-Modelo

modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")


In [ ]:

# Combinação de vetores por soma
import numpy as np
def combinacao_vetores_por_soma_skipgram(palavras_numeros):
    vetor_resultante = np.zeros(300,)  # vetor de zeros com dimensão 300
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo_skipgram.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0"*len(pn)  # substitui números por "0", "00", "000", etc.
                vetor_resultante += modelo_skipgram.get_vector(pn)
            else:
                vetor_resultante += modelo.get_vector("unknown")     
    
    return vetor_resultante


# Criar função para matrix de vetores
def matriz_vetores_skipgram(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores_por_soma_skipgram(palavras_numeros)

    return matriz

matriz_vetores_treino_skipgram = matriz_vetores_skipgram(artigo_treino.title)
matriz_vetores_teste_skipgram = matriz_vetores_skipgram(artigo_teste.title)

# Previsão com skip-gram
LR_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)
label_prevista_skipgram = LR_skipgram.predict(matriz_vetores_teste_skipgram) 
CR_skipgram = classification_report(artigo_teste.category, label_prevista_skipgram)
print(CR_skipgram)


# Utilizando o spaCy

In [ ]:
import pandas as pd

In [ ]:
dados_treino = pd.read_csv('treino.csv')
dados_treino.sample(5)

In [ ]:
# Instalando o spaCy
!pip install -U spacy
!python -m spacy download pt_core_news_sm

In [ ]:
# Arquitetura, centralizada em duas estruturas de dados principais Doc e Vocab. 
import spacy
nlp = spacy.load("pt_core_news_sm")

In [ ]:
texto = "Rio de janeiro é  as !@# 2131321 cidade maravilhosa."
doc = nlp(texto)

In [ ]:
# DOC -> Conjunto de tokens
print(type(doc))

# Conhecendo o DOC
print(doc[0])
print(doc.ents)
print(doc[0].is_stop)
print(doc[1].is_stop)

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [ ]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)

In [ ]:
trata_textos(doc)

In [ ]:
from time import time 

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size=1000,
                                                        n_process=-1 #utiliza todos os núcleos do processador
                                                         )]

tf = (time() - t0)/60
print(f"Tempo de processamento: {tf:.2f} minutos")                                                        

In [ ]:
titulos_tratados = pd.DataFrame({
    "titulo": textos_tratados
})
titulos_tratados.head(5)

In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, #CBOW
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [ ]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level=logging.INFO)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)



In [ ]:
dir(w2v_modelo)

In [ ]:
w2v_modelo.corpus_count

In [ ]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30)

In [ ]:
w2v_modelo.wv.most_similar("google")

In [ ]:
w2v_modelo.wv.most_similar("messi")

In [ ]:
# Treinando o modelo Skip-gram
w2v_modelo_sg = Word2Vec(sg = 1, # Skip-gram
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs=30)                      

In [ ]:
w2v_modelo_sg.wv.most_similar("google")

In [ ]:
w2v_modelo.wv.most_similar("google")

In [ ]:
w2v_modelo_sg.wv.most_similar("gm")

In [ ]:
w2v_modelo.wv.most_similar("gm")

In [ ]:
w2v_modelo.wv.save_word2vec_format("modelos/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("modelos/modelo_skipgram.txt", binary=False)

In [ ]:
# Criar função para matrix de vetores
def matriz_vetores(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores_por_soma(palavras_numeros)

    return matriz

In [2]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors


w2v_modelo_cbow = KeyedVectors.load_word2vec_format("modelos/modelo_cbow.txt")
w2v_modelo_sg   = KeyedVectors.load_word2vec_format("modelos/modelo_skipgram.txt")
artigo_treino = pd.read_csv('treino.csv')
artigo_teste = pd.read_csv('teste.csv')

In [3]:
nlp = spacy.load("pt_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())    
    return tokens_validos

texto = "TExto Exemplo, 1234. mais uma pedaço!!!"
tokens = tokenizador(texto)
print(tokens)

['texto', 'pedaço']


In [4]:

# Combinação de vetores por soma
def combinacao_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros((1,300))  # vetor de zeros com dimensão 300
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass    
    
    return vetor_resultante

vetor_texto = combinacao_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)


(1, 300)


In [5]:
# Criar função para matrix de vetores
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)


(90000, 300)
(20513, 300)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classficador(modelo, x_treino, y_treino, x_teste, y_teste):
    LR = LogisticRegression(max_iter=800)
    LR.fit(x_treino, y_treino)
    label_prevista = LR.predict(x_teste)
    CR = classification_report(y_teste, label_prevista)
    print(CR)
    return LR

LR_cbow = classficador(w2v_modelo_cbow, 
                       matriz_vetores_treino_cbow,
                       artigo_treino.category,
                       matriz_vetores_teste_cbow,
                       artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.84      0.78      0.80      5867
       mundo       0.74      0.83      0.79      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.82      0.78      0.80     20513



In [8]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

LR_sg = classficador(w2v_modelo_sg, 
                       matriz_vetores_treino_sg,
                       artigo_treino.category,
                       matriz_vetores_teste_sg,
                       artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.81      0.71      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.89      0.25       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.76      0.84      0.80      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.79      0.80     20513



In [10]:
import pickle
with open("modelos/modelo_logistico_cbow.pkl", "wb") as f:
    pickle.dump(LR_cbow, f)

with open("modelos/modelo_logistico_sg.pkl", "wb") as f:
    pickle.dump(LR_sg, f)